
Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [112]:
import pandas as pd
import numpy as np
# Start coding here...
raw_data = pd.read_csv("bank_marketing.csv")
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

### Clients

In [113]:
raw_clients = raw_data[["client_id","age","job","marital","education","credit_default","mortgage"]]
raw_clients.head(2)

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no


In [114]:
##  Apply the cleanings
#job
raw_clients["job"] = raw_clients["job"].str.replace(".","_")
#education
raw_clients["education"] = raw_clients["education"].str.replace(".","_")
raw_clients["education"] = raw_clients["education"].replace("unknown",np.NaN)
# credit_default
d = {'yes': True, 'no': False,"unknown":np.NaN}
raw_clients["credit_default"] = raw_clients["credit_default"].map(d).astype(bool)
# mortgage
raw_clients["mortgage"] = raw_clients["mortgage"].map(d).astype(bool)

In [115]:
print(raw_clients.info())
raw_clients.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None


,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,True,False


In [116]:
client = raw_clients
client.to_csv("./client.csv",index=False)

### Campaign


In [117]:
# Split the data
raw_campaign = raw_data[["client_id","month", "day",  "number_contacts",
                     "contact_duration","previous_campaign_contacts", 
                     "previous_outcome", "cons_price_idx", 
                     "euribor_three_months", "campaign_outcome"]]

In [118]:
# Combine the month and day columns with the year 2022 and convert to datetime
raw_campaign['last_contact_date'] = pd.to_datetime('2022-' + raw_campaign['month'] + '-' + raw_campaign['day'].astype(str), format='%Y-%b-%d')
raw_campaign.drop(columns= ["month","day"],inplace=True)


In [119]:
# previous_outcome
d = {"failure":False,"success":True, "nonexistent":np.NaN}
raw_campaign["previous_outcome"] = raw_campaign.previous_outcome.replace(d).astype(bool)
# campaign_outcome
d = {'yes': True, 'no': False}
raw_campaign["campaign_outcome"] = raw_campaign.campaign_outcome.replace(d).astype(bool)
raw_campaign.head(2)

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,last_contact_date
0,0,1,261,0,True,93.994,4.857,False,2022-05-13
1,1,1,149,0,True,93.994,4.857,False,2022-05-19


In [120]:
campaign = raw_campaign
campaign.to_csv("campaign.csv",index=False)

### Economics

In [121]:
economics = raw_data[["client_id","cons_price_idx","euribor_three_months"]]
economics.head(2)

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857


In [122]:
economics.to_csv("economics.csv",index=False)